In [1]:
import numpy as np
import pandas as pd
import re

In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
!ls

sample_data  training.1600000.processed.noemoticon.csv.zip


In [3]:
!unzip training.1600000.processed.noemoticon.csv.zip

Archive:  training.1600000.processed.noemoticon.csv.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [4]:
df = pd.read_csv(
    "training.1600000.processed.noemoticon.csv",
    encoding="latin-1", header=None
)

In [5]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
cols = [
    "target",
    "id",
    "date",
    "query",
    "user",
    "text"
]
df = pd.read_csv(
    "training.1600000.processed.noemoticon.csv",
    encoding="latin-1",
    header=None,
    names=cols
)


In [7]:
df.head()

,target,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.target.unique()

array([0, 4])

In [9]:
df.shape

(1600000, 6)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   query   1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


##Want to build a binary sentiment classification

In [11]:
df=df[["text","target"]]
df.head()

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


#BASIC TEXT CLEANING

In [12]:
#Lower casing
df["text"]=df["text"].str.lower()

In [13]:
df.head()

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0
1,is upset that he can't update his facebook by ...,0
2,@kenichan i dived many times for the ball. man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [14]:
#Remove URLs
import re

df["text"]=df["text"].apply(lambda x: re.sub(r"http\S+|www\S+", "", str(x)))

In [15]:
df.head()

,text,target
0,@switchfoot - a that's a bummer. you shoulda...,0
1,is upset that he can't update his facebook by ...,0
2,@kenichan i dived many times for the ball. man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [16]:
#Remove @mentions. Usernames don’t carry sentiment.
df["text"]=df["text"].apply(lambda x: re.sub(r"@\w+", "", str(x)))

In [17]:
df.head()

,text,target
0,- a that's a bummer. you shoulda got david ...,0
1,is upset that he can't update his facebook by ...,0
2,i dived many times for the ball. managed to s...,0
3,my whole body feels itchy and like its on fire,0
4,"no, it's not behaving at all. i'm mad. why am...",0


In [18]:
#Remove "#". option1: just remove # symbol, keeps the whole text. option2:creates new features (hashtags) for analysis.
#here I will use option 1
df["text"]=df["text"].str.replace(r"#","",regex=True)

In [19]:
#Remove numbers
df["text"]=df["text"].apply(lambda x: re.sub(r"\d+", "", str(x)))

In [20]:
df.head()

,text,target
0,- a that's a bummer. you shoulda got david ...,0
1,is upset that he can't update his facebook by ...,0
2,i dived many times for the ball. managed to s...,0
3,my whole body feels itchy and like its on fire,0
4,"no, it's not behaving at all. i'm mad. why am...",0


In [21]:
!pip install emoji --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 15.8 MB/s eta 0:00:00


In [22]:
#Keep emojis and interpret them
import emoji

def emoji_to_text(text):
    return emoji.demojize(text)  # Converts 😀 to :grinning_face:

df['text'] = df['text'].apply(emoji_to_text)

In [23]:
df.head()

,text,target
0,- a that's a bummer. you shoulda got david ...,0
1,is upset that he can't update his facebook by ...,0
2,i dived many times for the ball. managed to s...,0
3,my whole body feels itchy and like its on fire,0
4,"no, it's not behaving at all. i'm mad. why am...",0


In [24]:
#Remove special characters & punctuation
df["text"]=df["text"].apply(lambda x: re.sub(r"[^a-z\s]", "", str(x)))

In [25]:
df.head()

,text,target
0,a thats a bummer you shoulda got david car...,0
1,is upset that he cant update his facebook by t...,0
2,i dived many times for the ball managed to sa...,0
3,my whole body feels itchy and like its on fire,0
4,no its not behaving at all im mad why am i he...,0


In [26]:
#Remove extra white spaces
df["text"]=df["text"].apply(lambda x: re.sub(r"\s+", " ", str(x)))

In [27]:
df["text"]=df["text"].str.strip()

In [28]:
df.head()

,text,target
0,a thats a bummer you shoulda got david carr of...,0
1,is upset that he cant update his facebook by t...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no its not behaving at all im mad why am i her...,0


In [29]:
# Remove all types of leading/trailing whitespace
df["text"] = df["text"].str.replace(r"^\s+|\s+$", "", regex=True)

In [30]:
df.head()

,text,target
0,a thats a bummer you shoulda got david carr of...,0
1,is upset that he cant update his facebook by t...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no its not behaving at all im mad why am i her...,0


In [31]:
# Check if any string starts or ends with whitespace
leading_space = df["text"].str.match(r"^\s")
trailing_space= df["text"].str.match(r"\s$")

print(leading_space)
print(trailing_space)

0          False
1          False
2          False
3          False
4          False
           ...  
1599995    False
1599996    False
1599997    False
1599998    False
1599999    False
Name: text, Length: 1600000, dtype: bool
0          False
1          False
2          False
3          False
4          False
           ...  
1599995    False
1599996    False
1599997    False
1599998    False
1599999    False
Name: text, Length: 1600000, dtype: bool


In [32]:
print(leading_space.any())
print(trailing_space.any())

False
False


#TOKEN LEVEL PREPROCESSING

In [33]:
#Science text has now "dont,cant" etc. words we need to handle them
!pip install contractions --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.0 MB/s eta 0:00:00


In [34]:
import contractions

df['text'] = df['text'].apply(lambda x: contractions.fix(x))

In [35]:
#Stopword Removal
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [36]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))

In [37]:
#Remove rare words

from collections import Counter

# Combine all words from the dataset
all_words = " ".join(df['text'].fillna("")).split()

# Count frequency of each word
word_counts = Counter(all_words)

# Define threshold for rare words
threshold = 5
rare_words = {word for word, count in word_counts.items() if count < threshold}

print(f"Number of rare words: {len(rare_words)}")

Number of rare words: 361410


In [38]:
def remove_rare(text):
    """
    Remove words that appear less than threshold times in the dataset.
    """
    if not isinstance(text, str):
        return ""
    return " ".join([w for w in text.split() if w not in rare_words])

In [39]:
df['text'] = df['text'].apply(remove_rare)

In [40]:
#Spell correction

In [41]:
!pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 13.0 MB/s eta 0:00:00


In [42]:
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

# Load default dictionary (words + frequencies)
dictionary_path = "frequency_dictionary_en_82_765.txt"  # Comes with SymSpell repo
# You can download from https://github.com/mammothb/symspell
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Optional: add slang/custom words to protect them from correction
custom_words = ['gr8', 'u', 'b4', 'lol', 'idk']

2025-12-20 15:57:36,252: E symspellpy.symspellpy] Dictionary file not found at frequency_dictionary_en_82_765.txt.
ERROR:symspellpy.symspellpy:Dictionary file not found at frequency_dictionary_en_82_765.txt.


In [43]:
for w in custom_words:
    sym_spell.create_dictionary_entry(w, 1000)  # high frequency prevents change

# Function to correct text using SymSpell
def correct_spelling_symspell(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""

    words = text.split()
    corrected_words = []

    for word in words:
        # Lookup word, top 1 suggestion
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)

    return " ".join(corrected_words)

# Apply to DataFrame
df['text'] = df['text'].fillna("")
# Use chunking for faster progress and avoid memory issues
chunk_size = 50000
for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    df.loc[start:end, 'text'] = df.loc[start:end, 'text'].apply(correct_spelling_symspell)

In [44]:
df.head()

,text,target
0,bummer shoulda lol david carr third day,0
1,upset cannot update facebook texting might gr8...,0
2,many times ball managed save rest lol bounds,0
3,whole body feels itchy like fire,0
4,behaving mad cannot see,0


In [45]:
#Deal with chat words

slang_dict = {
    "gr8": "great",
    "u": "you",
    "ur": "your",
    "b4": "before",
    "idk": "i do not know",
    "lol": "laughing out loud",
    "lmao": "laughing my ass off",
    "rofl": "rolling on the floor laughing",
    "smh": "shaking my head",
    "tbh": "to be honest",
    "omg": "oh my god",
    "btw": "by the way",
    "imo": "in my opinion",
    "fyi": "for your information",
    "thx": "thanks",
    "ty": "thank you",
    "np": "no problem",
    "ily": "i love you",
    "jk": "just kidding",
    "hbu": "how about you",
    "bff": "best friends forever",
    "rn": "right now",
    "afaik": "as far as i know",
    "idc": "i do not care",
    "smth": "something",
    "bc": "because",
    "pls": "please",
    "plz": "please",
    "w/": "with",
    "w/o": "without",
    "tho": "though",
    "ya": "yeah",
    "yea": "yes",
    "cya": "see you",
    "l8r": "later"
}

In [46]:
def replace_slang(text):
    """
    Replaces slang/chat words in text with their standard meaning.
    """
    if not isinstance(text, str):
        return ""

    words = text.split()
    # Replace slang if in dictionary
    return " ".join([slang_dict.get(w, w) for w in words])

In [47]:
df['text'] = df['text'].apply(replace_slang)

In [48]:
#pos tagging + lemmatization

!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [49]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

In [50]:
def spacy_lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-"])

In [51]:
texts = df['text'].tolist()

lemmatized_texts = []
for doc in nlp.pipe(texts, batch_size=1000):
    lemmatized_texts.append(
        " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-"])
    )

df['text'] = lemmatized_texts

In [52]:
df.to_csv("tweets_lemmatized.csv", index=False)

In [53]:
del texts
del lemmatized_texts
nlp = None

In [54]:
import gc
gc.collect()

73662

In [21]:
df = pd.read_csv("tweets_lemmatized.csv")
df.head()

,text,target
0,bummer shoulda laugh out loud david carr third...,0
1,upset can not update facebook texting might gr...,0
2,many time ball manage save rest laugh out loud...,0
3,whole body feel itchy like fire,0
4,behave mad can not see,0


In [22]:
#Negation Marking

NEGATION_WORDS = {
    "not", "no", "never", "none", "cannot",
    "neither", "nor"
}

In [23]:
def negation_marking(text):
    words = text.split()
    result = []
    negate = False

    for w in words:
        if w in NEGATION_WORDS:
            negate = True
            result.append(w)
        elif negate:
            result.append(w + "_NEG")
        else:
            result.append(w)

        # stop negation on sentence boundary
        if w in {".", "!", "?"}:
            negate = False

    return " ".join(result)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1587075 non-null  object
 1   target  1600000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [25]:
df['text'] = df['text'].fillna("")
df['text'] = df['text'].astype(str)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1600000 non-null  object
 1   target  1600000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [27]:
df['text'] = df['text'].apply(negation_marking)

In [28]:
df.head()

,text,target
0,bummer shoulda laugh out loud david carr third...,0
1,upset can not update_NEG facebook_NEG texting_...,0
2,many time ball manage save rest laugh out loud...,0
3,whole body feel itchy like fire,0
4,behave mad can not see_NEG,0


In [29]:
#repeated character normalization or character de-elongation.

#Reduce 3+ repeated letters to 2
def reduce_repeated_characters(text):
    """
    Reduces any character repeated 3 or more times to 2.
    Example: 'loooove' -> 'loove', 'soooo' -> 'soo'
    """
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

In [30]:
df['text'] = df['text'].fillna("").apply(reduce_repeated_characters)

In [31]:
df.head()

,text,target
0,bummer shoulda laugh out loud david carr third...,0
1,upset can not update_NEG facebook_NEG texting_...,0
2,many time ball manage save rest laugh out loud...,0
3,whole body feel itchy like fire,0
4,behave mad can not see_NEG,0


In [32]:
df['text']=df["text"].str.strip()

In [33]:
df.head()

,text,target
0,bummer shoulda laugh out loud david carr third...,0
1,upset can not update_NEG facebook_NEG texting_...,0
2,many time ball manage save rest laugh out loud...,0
3,whole body feel itchy like fire,0
4,behave mad can not see_NEG,0


In [34]:
#Tokenization

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')  # keeps only words

df['tokens'] = df['text'].apply(tokenizer.tokenize)

In [35]:
df.head()

,text,target,tokens
0,bummer shoulda laugh out loud david carr third...,0,"[bummer, shoulda, laugh, out, loud, david, car..."
1,upset can not update_NEG facebook_NEG texting_...,0,"[upset, can, not, update_NEG, facebook_NEG, te..."
2,many time ball manage save rest laugh out loud...,0,"[many, time, ball, manage, save, rest, laugh, ..."
3,whole body feel itchy like fire,0,"[whole, body, feel, itchy, like, fire]"
4,behave mad can not see_NEG,0,"[behave, mad, can, not, see_NEG]"


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1600000 non-null  object
 1   target  1600000 non-null  int64 
 2   tokens  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [37]:
df["target"].unique()

array([0, 4])

In [38]:
df["target"].value_counts()

,count
target,
0,800000
4,800000


In [22]:
df.to_csv("Final_tweets.csv",index=False)

In [2]:
df = pd.read_csv("Final_tweets.csv")
df.head()

,text,target,tokens
0,bummer shoulda laugh out loud david carr third...,0,"['bummer', 'shoulda', 'laugh', 'out', 'loud', ..."
1,upset can not update_NEG facebook_NEG texting_...,0,"['upset', 'can', 'not', 'update_NEG', 'faceboo..."
2,many time ball manage save rest laugh out loud...,0,"['many', 'time', 'ball', 'manage', 'save', 're..."
3,whole body feel itchy like fire,0,"['whole', 'body', 'feel', 'itchy', 'like', 'fi..."
4,behave mad can not see_NEG,0,"['behave', 'mad', 'can', 'not', 'see_NEG']"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1587075 non-null  object
 1   target  1600000 non-null  int64 
 2   tokens  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [4]:
df['text'] = df['text'].fillna('')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1600000 non-null  object
 1   target  1600000 non-null  int64 
 2   tokens  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


#Feature representation or text representation

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
#TF-IDF. I will use it for this project

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = tfidf.fit_transform(df['text'])
y = df['target']

In [19]:
X.shape

(1600000, 5000)

In [6]:
# Adding extra features

df['tweet_length'] = df['text'].apply(len) #Tweet length

In [7]:
df.head()

,text,target,tokens,tweet_length
0,bummer shoulda laugh out loud david carr third...,0,"['bummer', 'shoulda', 'laugh', 'out', 'loud', ...",50
1,upset can not update_NEG facebook_NEG texting_...,0,"['upset', 'can', 'not', 'update_NEG', 'faceboo...",119
2,many time ball manage save rest laugh out loud...,0,"['many', 'time', 'ball', 'manage', 'save', 're...",52
3,whole body feel itchy like fire,0,"['whole', 'body', 'feel', 'itchy', 'like', 'fi...",31
4,behave mad can not see_NEG,0,"['behave', 'mad', 'can', 'not', 'see_NEG']",26


In [8]:
#Combine TF-IDF X with tweet_length column

In [11]:
from scipy.sparse import hstack
import numpy as np
from scipy.sparse import csr_matrix

In [12]:
# Convert tweet_length to 2D array
tweet_length_feature = df['tweet_length'].values.reshape(-1, 1)

In [18]:
X_extra = csr_matrix(tweet_length_feature)

In [25]:
# hstack expects both matrices; convert numerical features to sparse

X_combined = hstack([X, X_extra])

In [26]:
X_combined.shape

(1600000, 5001)

In [16]:
y.head()

,target
0,0
1,0
2,0
3,0
4,0


In [17]:
# Split the dataset into training & test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=99
)

#Model building

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import accuracy_score, classification_report,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [19]:
#========Baseline First model========================================
#Naive bayes

nb = MultinomialNB(alpha=0.1)
nb.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [20]:
y_pred_nb = nb.predict(X_test)

In [23]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_nb == 4, 1, 0)  # y_pred_nb is a NumPy array

f1 = f1_score(y_test_bin, y_pred_bin)
print("F1-score:", f1)

F1-score: 0.7358769521301047


In [38]:
#===========Baseline second model===================
#Logistic regression

lr = LogisticRegression(
    solver="saga",
    penalty="l2",
    C=1.0,
    max_iter=1000,
    n_jobs=-1,
    random_state=42
)
lr.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42, solver='saga')

In [39]:
y_pred_lr = lr.predict(X_test)

In [40]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_lr == 4, 1, 0)  # y_pred_lr is a NumPy array

f1 = f1_score(y_test_bin, y_pred_bin)
print("F1-score:", f1)

F1-score: 0.7416042102438697


In [41]:
#===========Third baseline model===========================
#Support vector machine
svc = LinearSVC(
    C=1.0,               # baseline, can tune later
    max_iter=1000,        # increase if needed
    random_state=42
)
svc.fit(X_train, y_train)

LinearSVC(random_state=42)

In [42]:
y_pred_svc = svc.predict(X_test)

In [43]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_svc == 4, 1, 0)  # y_pred_lr is a NumPy array

f1 = f1_score(y_test_bin, y_pred_bin)
print("F1-score:", f1)

F1-score: 0.7600913614980865


In [27]:
#============Hyperparameter tuning=======================================
X_tune = X_train[:50000]
y_tune = y_train[:50000]

In [28]:
# Logisti Regression
param_dist = {"C": [0.01, 0.1, 1, 5, 10]}

lr = LogisticRegression(
    solver="saga",
    penalty="l2",
    max_iter=300,
    n_jobs=-1,
    random_state=42
)

In [29]:
random_search = RandomizedSearchCV(
    lr,
    param_dist,
    n_iter=5,
    cv=2,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

random_search.fit(X_tune, y_tune)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RandomizedSearchCV(cv=2,
                   estimator=LogisticRegression(max_iter=300, n_jobs=-1,
                                                random_state=42,
                                                solver='saga'),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'C': [0.01, 0.1, 1, 5, 10]},
                   scoring='f1', verbose=2)

In [30]:
best_C = random_search.best_params_["C"]

In [31]:
best_C

0.01

In [32]:
# Training on full dataset
final_lr = LogisticRegression(
    solver="saga",
    penalty="l2",
    C=best_C,   #0.01
    max_iter=1000,
    n_jobs=-1,
    random_state=42
)

final_lr.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.01, max_iter=1000, n_jobs=-1, random_state=42,
                   solver='saga')

In [35]:
y_pred_lr = final_lr.predict(X_test)

In [36]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_lr == 4, 1, 0)  # y_pred_lr is a NumPy array

f1 = f1_score(y_test_bin, y_pred_bin)
print("F1-score:", f1)

F1-score: 0.7390557966087224


In [44]:
# Support vector
param_dist = {"C": [0.01, 0.1, 1, 5, 10]}

random_search_svc = RandomizedSearchCV(
    LinearSVC(max_iter=1000, random_state=42),
    param_distributions=param_dist,
    n_iter=5,
    cv=2,
    scoring="f1",
    verbose=2,
    n_jobs=-1
)

random_search_svc.fit(X_tune, y_tune)
best_C = random_search_svc.best_params_["C"]

Fitting 2 folds for each of 5 candidates, totalling 10 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


In [45]:
best_C

0.01

In [49]:
#Train on full dataset
svc_final = LinearSVC(
    C=0.01,               # best_C
    max_iter=1000,        # increase if needed
    random_state=42
)
svc_final.fit(X_train, y_train)

LinearSVC(C=0.01, random_state=42)

In [50]:
y_pred_svcf = svc_final.predict(X_test)

In [51]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_svcf == 4, 1, 0)  # y_pred_lr is a NumPy array

f1 = f1_score(y_test_bin, y_pred_bin)
print("F1-score:", f1)

F1-score: 0.7584220436057353


So till now the baseline SVC model gave highest accuracy

In [16]:
#===============Again doing feature tuning========================
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=5,
    max_df=0.9,
    sublinear_tf=True
)
X = tfidf.fit_transform(df['text'])
y = df['target']

In [19]:
#Feature combine
X_combined = hstack([X, X_extra])

In [20]:
X_combined.shape

(1600000, 261707)

In [21]:
# Split the dataset into training & test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=99
)

In [32]:
#========Training baseline models==============

In [33]:
#Naive bayes

nb = MultinomialNB(alpha=0.1)

nb.fit(X_train, y_train)

y_pred_nb = nb.predict(X_test)

In [34]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_nb == 4, 1, 0)  # y_pred_lr is a NumPy array

In [35]:
print("Naive Bayes F1-score:", f1_score(y_test_bin, y_pred_bin))
print(classification_report(y_test_bin, y_pred_bin))

Naive Bayes F1-score: 0.7528082551664744
              precision    recall  f1-score   support

           0       0.76      0.74      0.75    160305
           1       0.75      0.76      0.75    159695

    accuracy                           0.75    320000
   macro avg       0.75      0.75      0.75    320000
weighted avg       0.75      0.75      0.75    320000



In [39]:
#Logistic Regression
lr = LogisticRegression(
    solver="saga",
    penalty="l2",
    C=1.0,
    max_iter=400,
    n_jobs=-1,
    random_state=42,
    tol=1e-3
)

lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [40]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_lr == 4, 1, 0)  # y_pred_lr is a NumPy array

In [41]:
print("Logistic Regression F1-score:", f1_score(y_test_bin, y_pred_bin))
print(classification_report(y_test_bin, y_pred_bin))

Logistic Regression F1-score: 0.719133453892341
              precision    recall  f1-score   support

           0       0.72      0.70      0.71    160305
           1       0.71      0.73      0.72    159695

    accuracy                           0.72    320000
   macro avg       0.72      0.72      0.72    320000
weighted avg       0.72      0.72      0.72    320000



In [36]:
#Linear SVC
svc = LinearSVC(
    C=1.0,
    max_iter=600,
    random_state=42,
    tol=1e-3
)

svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

In [37]:
# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_svc == 4, 1, 0)  # y_pred_lr is a NumPy array

In [38]:
print("LinearSVC F1-score:", f1_score(y_test_bin, y_pred_bin))
print(classification_report(y_test_bin, y_pred_bin))

LinearSVC F1-score: 0.7748561635451096
              precision    recall  f1-score   support

           0       0.78      0.75      0.77    160305
           1       0.76      0.79      0.77    159695

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [46]:
# Linear SVC gave highest F1 score.
# TUNING hyperparameter for SVC.

param_dist = {"C": [0.01, 0.1, 5, 10]}

X_tune = X_train[:50000]  # small subset
y_tune = y_train[:50000]

from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC

random_search = RandomizedSearchCV(
    LinearSVC(max_iter=1500, random_state=42),
    param_distributions=param_dist,
    n_iter=5,
    cv=2,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

random_search.fit(X_tune, y_tune)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 4 candidates, totalling 8 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomizedSearchCV(cv=2, estimator=LinearSVC(max_iter=1500, random_state=42),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'C': [0.01, 0.1, 5, 10]}, scoring='f1',
                   verbose=2)

In [47]:
best_C = random_search.best_params_["C"]
print("Best C:", best_C)

Best C: 0.01


In [48]:
#Training on full dataset
svc = LinearSVC(
    C=0.01,
    max_iter=600,
    random_state=42,
    tol=1e-3
)

svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_svc == 4, 1, 0)  # y_pred_lr is a NumPy array

print("LinearSVC F1-score:", f1_score(y_test_bin, y_pred_bin))
print(classification_report(y_test_bin, y_pred_bin))

LinearSVC F1-score: 0.7690404815870927
              precision    recall  f1-score   support

           0       0.78      0.74      0.76    160305
           1       0.75      0.79      0.77    159695

    accuracy                           0.76    320000
   macro avg       0.76      0.76      0.76    320000
weighted avg       0.76      0.76      0.76    320000



Applying stacking method

In [22]:
from sklearn.ensemble import StackingClassifier

In [24]:
X_stack = X_train[:150000]
y_stack = y_train[:150000]

In [25]:
# Base learners
base_learners = [
    ('nb', MultinomialNB(alpha=0.1)),
    ('lr', LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1, random_state=42)),
    ('svc', LinearSVC(C=1.0, max_iter=1000, random_state=42))
]

# Meta-learner
meta_learner = LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42)

# Stacking classifier
stack_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=3,
    n_jobs=-1,
    passthrough=False     # IMPORTANT: reduces memory
)


In [26]:
stack_model.fit(X_stack, y_stack)

StackingClassifier(cv=3,
                   estimators=[('nb', MultinomialNB(alpha=0.1)),
                               ('lr',
                                LogisticRegression(max_iter=1000, n_jobs=-1,
                                                   random_state=42,
                                                   solver='saga')),
                               ('svc', LinearSVC(random_state=42))],
                   final_estimator=LogisticRegression(max_iter=1000, n_jobs=-1,
                                                      random_state=42),
                   n_jobs=-1)

In [27]:
# Predict & evaluate
y_pred_stack = stack_model.predict(X_test)

# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_stack == 4, 1, 0)


print(classification_report(y_test_bin, y_pred_bin))

print("Stacked F1-score:", f1_score(y_test_bin, y_pred_bin))

              precision    recall  f1-score   support

           0       0.82      0.57      0.67    160305
           1       0.67      0.87      0.76    159695

    accuracy                           0.72    320000
   macro avg       0.74      0.72      0.71    320000
weighted avg       0.74      0.72      0.71    320000

Stacked F1-score: 0.7570528354125361


In [28]:
# Stacking model with Linear SVC meta-learner
base_learners = [
    ("nb", MultinomialNB(alpha=0.1)),
    ("lr", LogisticRegression(
        solver="saga",
        max_iter=500,
        C=1.0,
        n_jobs=-1,
        random_state=42
    )),
    ("svc", LinearSVC(
        C=1.0,
        max_iter=1000,
        random_state=42
    ))
]

In [29]:
meta_learner = LinearSVC(
    C=0.1,              # STRONG regularization
    max_iter=1000,
    random_state=42
)

In [30]:
stack_svc_meta = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=3,               # keep LOW
    n_jobs=-1,
    passthrough=False   # IMPORTANT for memory
)

In [31]:
stack_svc_meta.fit(X_stack, y_stack)

StackingClassifier(cv=3,
                   estimators=[('nb', MultinomialNB(alpha=0.1)),
                               ('lr',
                                LogisticRegression(max_iter=500, n_jobs=-1,
                                                   random_state=42,
                                                   solver='saga')),
                               ('svc', LinearSVC(random_state=42))],
                   final_estimator=LinearSVC(C=0.1, random_state=42),
                   n_jobs=-1)

In [32]:
y_pred_stack = stack_svc_meta.predict(X_test)

# Convert 4 → 1 for binary labels
y_test_bin = y_test.replace(4, 1)  # y_test is a pandas Series
y_pred_bin = np.where(y_pred_stack == 4, 1, 0)


print(classification_report(y_test_bin, y_pred_bin))

print("Stacked F1-score:", f1_score(y_test_bin, y_pred_bin))

              precision    recall  f1-score   support

           0       0.85      0.43      0.57    160305
           1       0.62      0.93      0.74    159695

    accuracy                           0.68    320000
   macro avg       0.74      0.68      0.66    320000
weighted avg       0.74      0.68      0.66    320000

Stacked F1-score: 0.7418244190391169
